# Non Planar Reconstruction (NPR) and Cross-Sectional View of a Vessel

aseresti@github.com

The goal of this script is to take the centerline of the vessel in .pth format (SimVascular Path files) and extract the NPR and cross-sections of the vessel at a given point.


### Import frameworks and define necessary functions

In [159]:
import vtk
import xml.etree.ElementTree as ET
import os
import numpy as np

def ReadVTPFile(FileName):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader

def CutPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    return Slice.GetOutput()

def ReadVTIFile(FileName):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader.GetOutput()

def WriteVTPFile(FileName,Data):
    writer=vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(FileName)
    writer.SetInputData(Data)
    writer.Update()

def SphereClip(volume_image,center,radius):
    sphere = vtk.vtkSphere()
    sphere.SetCenter(center)
    sphere.SetRadius(radius)

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume_image)
    clipper.SetClipFunction(sphere)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def PlaneClip(volume,center, a):
    box = vtk.vtkBox()
    box.SetBounds(
        center[0] - a, center[0] + a,
        center[1] - a, center[1] + a,
        center[2] - a, center[2] + a
    )

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume)
    clipper.SetClipFunction(box)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def gradient_filter(vtk_image):
    gradient_filter = vtk.vtkImageGradient()
    gradient_filter.SetInputData(vtk_image)
    gradient_filter.SetDimensionality(3)
    gradient_filter.Update()

    return gradient_filter.GetOutput()
    
def define_borders(gradient_image):
    magnitude_filter = vtk.vtkImageMagnitude()
    magnitude_filter.SetInputData(gradient_image)
    magnitude_filter.Update()
    
    return magnitude_filter.GetOutput()

def ExtractSurface(UnstrcturedGrid):
    geometry_filter = vtk.vtkGeometryFilter()
    geometry_filter.SetInputData(UnstrcturedGrid)
    geometry_filter.Update()

    return geometry_filter.GetOutput()

def WriteVTUFile(FileName,Data):
	writer=vtk.vtkXMLUnstructuredGridWriter()
	writer.SetFileName(FileName)
	writer.SetInputData(Data)
	writer.Update()

def clip_polydata_with_plane(polydata, origin, normal, inside_out=False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(origin)
    plane.SetNormal(normal)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(polydata)
    clipper.SetClipFunction(plane)

    if inside_out:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()
    
    clipper.Update()

    return clipper.GetOutput()

### Enter the path to the SimVascular pathfile of the vessel and VTI image

In [162]:
pathline_file = "/Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU03A/SimVascular_Edgard/Paths/L_Diag1_0.pth"
ImagePath = "/Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU03A/SimVascular_Edgard/Images/ct.vti"

### Read the pathline

In [163]:
with open(pathline_file, "r") as path:
    #path.readlines()
    tree = ET.parse(path)
root = tree.getroot()

direction_points = []
for direction_point in root.findall(".//path_point/tangent"):
    x = float(direction_point.attrib['x'])
    y = float(direction_point.attrib['y'])
    z = float(direction_point.attrib['z'])
    direction_points.append((x,y,z))

path_points = []
for path_point in root.findall(".//path_point/pos"):
    x = float(path_point.attrib['x'])
    y = float(path_point.attrib['y'])
    z = float(path_point.attrib['z'])
    path_points.append((x,y,z))

path_normals = []
for normal in root.findall(".//path_point/rotation"):
    x = float(normal.attrib['x'])
    y = float(normal.attrib['y'])
    z = float(normal.attrib['z'])
    path_normals.append((x,y,z))

NPoints = len(path_points)
print("the number of points in the centerline is:", NPoints)

binormal = []
for i in range(NPoints):
    binormal_ = np.cross(np.array(direction_points[i]),np.array(path_normals[i]))
    binormal_ /= np.linalg.norm(binormal_)
    binormal.append(binormal_)
    

the number of points in the centerline is: 113


### Read the image and Extract the gradient of the CTA

In [164]:
Volume = ReadVTIFile(ImagePath)
Volume_Gradient = define_borders(gradient_filter(Volume))

### Transform the vessel from 3D to 2D

 - LAD: in XY plane, flatten Z, note: for proximal Left main and LAD aroun the heart apex doesn't work well.
 - RCA/LCx: somewhere in between XY and XZ. a normal of (0, $\frac{\sqrt(2)}{2}$, $\frac{\sqrt(2)}{2}$) is suggested. Also, the desired normal value could be identified on Paraview.
 - Diag: in XY plane, flatten Z.
 - Intermedius/ramus: in XZ Plane, flatten Y. 

In [165]:
projection_matrix_XY = (   #Flatten Z
    1, 0, 0, 0,
    0, 1, 0, 0,
    0, 0, 0, 0,
    0, 0, 0, 1
)

projection_matrix_XZ = (   #Flatten Y
    1, 0, 0, 0,
    0, 0, 0, 0,
    0, 0, 1, 0,
    0, 0, 0, 1
)

projection_matrix_YZ = (   #Flatten X
    0, 0, 0, 0,
    0, 1, 0, 0,
    0, 0, 1, 0,
    0, 0, 0, 1
)

import numpy as np

#normal = np.array([0, ((2)**0.5)/2, ((2)**0.5)/2])
normal = np.array([-0.2762, -0.3362, 0.9003])
I = np.eye(3)
P = I - np.outer(normal,normal)
projection_matrix = np.zeros((4,4))
projection_matrix[:3,:3] = P
projection_matrix[3,3] = 1

projection_matrix = tuple(projection_matrix.flatten())


new_centerline = [(0,0,0)]
a_ = 0
tangent =[]
for i in range(1,NPoints):
    p = np.array(path_points[i])
    p_ = np.array(path_points[i-1])
    a = np.linalg.norm(p - p_)/2
    a_ += a 
    new_centerline.append((0,-(a_),0))

### Select an appropriate projection matrix based on the orientation of the vessel

In [166]:
projection_matrix = projection_matrix_XY

transform = vtk.vtkTransform()
transform.SetMatrix(projection_matrix)
transform.Update()

def Transform3Dto2D(section):
    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(section)
    transform_filter.SetTransform(transform)
    transform_filter.Update()

    return transform_filter.GetOutput()


def RotatePlane(plane, T1, N1, O1, O2):
    
    T2 = np.array([0, 1, 0])
    N2 = np.array([0, 0, 1])
    B1 = np.cross(N1, T1)
    B2 = np.cross(N2, T2)
    
    I = np.eye(3)
    P = I - np.outer(B2,B2)
    projection_matrix = np.zeros((4,4))
    projection_matrix[:3,:3] = P
    projection_matrix[3,3] = 1

    projection_matrix = tuple(projection_matrix.flatten())


    transform = vtk.vtkTransform()
    transform.Translate(-O1[0], -O1[1], -O1[2])
    transform.SetMatrix(projection_matrix)

    transform.Translate(O2[0], O2[1], O2[2])
    transform.Update()

    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(plane)
    transform_filter.SetTransform(transform)
    transform_filter.Update()

    return transform_filter.GetOutput()

### Extract the vessel NPR

In [167]:
NPoints = len(path_points)
print("the number of points in the centerline is:", NPoints)

append_filter_npr3d = vtk.vtkAppendPolyData()
append_filter_npr2d = vtk.vtkAppendPolyData()
#append_vessel_volume = vtk.vtkAppendFilter()

p = path_points[5]
p_ = path_points[0]
a = (((p[0]-p_[0])**2 + (p[1]-p_[1])**2 + (p[2]-p_[2])**2)**0.5)/2

a = 10

step = 1
for i in range(13, NPoints-1, step):
    #VesselVolume = PlaneClip(Volume, path_points[i],a)
    npr3d_ = CutPlane(PlaneClip(Volume, path_points[i],a), path_points[i], binormal[i])
    npr3d_ = clip_polydata_with_plane(npr3d_,path_points[i-1], direction_points[i])
    npr3d_ = clip_polydata_with_plane(npr3d_,path_points[i+1], direction_points[i], True)

    append_filter_npr3d.AddInputData(npr3d_)
    target_plane = RotatePlane(npr3d_, direction_points[i], binormal[i], np.array(path_points[i]), np.array(new_centerline[i]))
    append_filter_npr2d.AddInputData(target_plane)
    #append_vessel_volume.AddInputData(VesselVolume)
#print(a)
append_filter_npr3d.Update()
append_filter_npr2d.Update()
NPR3D = append_filter_npr3d.GetOutput()
NPR2D = append_filter_npr2d.GetOutput()#Transform3Dto2D(NPR3D)
#append_vessel_volume.Update()


the number of points in the centerline is: 113


In [174]:
# TEST


import numpy as np

def points_to_vtp(points, output_vtp):
    # Create VTK points
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)
    
    # Create a polyline
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(len(points))
    for i in range(len(points)):
        polyline.GetPointIds().SetId(i, i)
    
    # Create a cell array to store the polyline
    cells = vtk.vtkCellArray()
    cells.InsertNextCell(polyline)

    # Create a polydata object
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(vtk_points)
    polydata.SetLines(cells)

    # Write to a .vtp file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_vtp)
    writer.SetInputData(polydata)
    writer.Write()


NPoints = len(path_points)
print("the number of points in the centerline is:", NPoints)

def transform_polydata(polydata, original_orgin, new_origin, original_normal, target_normal):
    #translation_vector = np.array(new_origin) - np.array(original_orgin)
    original_normal = original_normal / np.linalg.norm(original_normal)
    #target_normal = target_normal / np.linalg.norm(target_normal)

    #target_normal = target_normal/np.linalg.norm(target_normal)
    rotation_axis = np.cross(original_normal, target_normal)
    rotation_angle = np.arccos(np.dot(original_normal, target_normal)) * (180.0/np.pi)
    #print(rotation_angle)
    #print(rotation_axis)
    transform = vtk.vtkTransform()
    transform.Translate(-original_orgin[0], -original_orgin[1], -original_orgin[2])
    if np.linalg.norm(rotation_axis) > 1e-6:
        transform.RotateWXYZ(rotation_angle, *rotation_axis)

    transform.Translate(new_origin[0], new_origin[1], new_origin[2])#translation_vector)
    
    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(polydata)
    transform_filter.SetTransform(transform)
    transform_filter.Update()
    
    return transform_filter.GetOutput()

stack_1 = vtk.vtkAppendPolyData()
stack_2 = vtk.vtkAppendPolyData()


new_centerline = [(0,0,0)]
a_ = 0
tangent =[]
for i in range(1,NPoints):
    p = np.array(path_points[i])
    p_ = np.array(path_points[i-1])
    a = np.linalg.norm(p - p_)#(((p[0]-p_[0])**2 + (p[1]-p_[1])**2 + (p[2]-p_[2])**2)**0.5)
    a_ += a 
    new_centerline.append((0,-(a_),0))
    tangent_ = np.array(new_centerline[i])-np.array(new_centerline[i-1])
    tangent_ /= np.linalg.norm(tangent_)
    tangent.append(tangent_)


def flatten_polydata(polydata, axis=2, value=0.0):
    new_points = vtk.vtkPoints()
    num_points = polydata.GetNumberOfPoints()
    
    for i in range(num_points):
        pt = list(polydata.GetPoint(i))
        pt[axis] = value  # Set the desired coordinate to a constant
        new_points.InsertNextPoint(pt)
    
    # Create a new polydata with the modified points
    flattened_polydata = vtk.vtkPolyData()
    flattened_polydata.ShallowCopy(polydata)
    flattened_polydata.SetPoints(new_points)
    
    return flattened_polydata

def ComputePCA (plane):
    points = np.array([plane.GetPoint(i) for i in range(plane.GetNumberOfPoints())])
    covariance_matrix = np.cov(points.T)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, sorted_indices]
    major_axis = eigenvectors[:,0]
    minor_axis = eigenvectors[:,1]
    normal = eigenvectors[:,2]
    projected_major = np.dot(points, major_axis)
    projected_minor = np.dot(points, minor_axis)

    height = np.max(projected_major) - np.min(projected_major)
    width = np.max(projected_minor) - np.min(projected_minor)

    num_points_major = len(np.unique(np.round(projected_major, decimals=4)))
    num_points_minor = len(np.unique(np.round(projected_minor, decimals=4)))

    #res_major = height/(num_points_major - 1)
    #res_minor = width/(num_points_minor - 1)

    return height, width, num_points_major, num_points_minor

def CreatePlane(center, width, height, Npoints):
    origin = (center[0] - height/2, center[1] - width/2, center[2])
    point1 = (center[0] + height/2, center[1] - width/2, center[2])
    point2 = (center[0] - height/2, center[1] + width/2, center[2])

    plane = vtk.vtkPlaneSource()
    plane.SetOrigin(*origin)
    plane.SetPoint1(*point1)
    plane.SetPoint2(*point2)
    xR = int(np.sqrt(Npoints*height/width)) - 1
    yR = int(np.sqrt(Npoints*width/height)) - 1
    plane.SetResolution(xR, yR)
    #plane.SetXResolution(x_npoints)
    #plane.SetYResolution(y_npoints)
    plane.Update()
    
    return plane.GetOutput()

def CopyPlane(original_plane, target_plane):
    source_scalar = original_plane.GetPointData().GetScalars()

    target_scalar = vtk.vtkFloatArray()
    target_scalar.SetNumberOfComponents(1)
    target_scalar.SetName("CopiedScalars")

    for i in range(target_plane.GetNumberOfPoints()):
        target_scalar.InsertNextValue(source_scalar.GetTuple1(i))

    target_plane.GetPointData().SetScalars(target_scalar)

    return target_plane

def RotatePlane(plane, T1, B1, O1, O2, T2):
    
    #T2 = np.array([0, 1, 0])
    #N2 = np.array([0, 0, 1])
    B2 = np.array([1, 0, 0])

    
    transform = vtk.vtkTransform()
    transform.Translate(-O1[0], -O1[1], -O1[2])

    #transform.RotateZ(rotation_around_normal)
    #transform.RotateX(rotation_around_binormal)
    

    #transform.Translate(O2[0], O2[1], O2[2])
    transform.Update()

    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(plane)
    transform_filter.SetTransform(transform)
    transform_filter.Update()
    Plane1 = transform_filter.GetOutput()

    


    #transform.RotateZ(rotation_around_normal)
    #transform.RotateX(rotation_around_binormal)
    rotation_axis = np.cross(B1, B2)
    rotation_angle = np.arccos(np.dot(B1, B2)) * (180.0 / np.pi)


    transform = vtk.vtkTransform()
    transform.RotateWXYZ(rotation_angle, *rotation_axis)
    transform.Update()

    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(Plane1)
    transform_filter.SetTransform(transform)
    transform_filter.Update()
    Plane2 = transform_filter.GetOutput()

    p0 = np.array(Plane2.GetPoint(0))
    p1 = np.array(Plane2.GetPoint(1))

    tangent = p1 - p0
    tangent /= np.linalg.norm(tangent)
    T2 = np.array([0,1,0])

    t_pr = tangent - np.dot(tangent,T2)*T2
    t_pr /= np.linalg.norm(t_pr)

    angle = np.arccos(np.dot(tangent, t_pr))
    angle = np.degrees(angle)

    transform = vtk.vtkTransform()
    transform.RotateX(angle)
    transform.Update()

    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(Plane2)
    transform_filter.SetTransform(transform)
    transform_filter.Update()
    Plane3 = transform_filter.GetOutput()


    
    transform = vtk.vtkTransform()
    transform.Translate(O2[0], O2[1], O2[2])
    transform.Update()

    transform_filter = vtk.vtkTransformFilter()
    transform_filter.SetInputData(Plane3)
    transform_filter.SetTransform(transform)
    transform_filter.Update()
    Plane4 = transform_filter.GetOutput()

    return Plane4


OutputFolder = os.path.dirname(pathline_file)
VesselName = os.path.splitext(os.path.basename(pathline_file))[0]
OutputFolder = os.path.join(OutputFolder,VesselName)

points_to_vtp(new_centerline, f"{OutputFolder}/{VesselName}_2D.vtp")
step = 2

for i in range(1, 50, step):

    a = 10
    npr3d_ = CutPlane(PlaneClip(Volume, path_points[i],a), path_points[i], binormal[i])
    npr3d_ = clip_polydata_with_plane(npr3d_,path_points[i-1], direction_points[i])
    npr3d_ = clip_polydata_with_plane(npr3d_,path_points[i+1], direction_points[i], True)
    #npr3d_flatten = flatten_polydata(npr3d_)
    output_path_plane = f"{OutputFolder}/npr_3d_{i}.vtp"
    WriteVTPFile(output_path_plane,npr3d_)
    #height, width, res_major, res_minor = ComputePCA(npr3d_)
    #print(height, width, res_major, res_minor)
    #target_plane_ = CreatePlane(new_centerline[i], width, height, npr3d_.GetNumberOfPoints())
    #print(target_plane_.GetNumberOfPoints(), npr3d_.GetNumberOfPoints())
    #target_plane = CopyPlane(npr3d_, target_plane_)
    target_plane= RotatePlane(npr3d_, direction_points[i], binormal[i], np.array(path_points[i]), np.array(new_centerline[i]), tangent[i])
    print(i, direction_points[i], tangent[i])
    print(np.degrees(np.arccos(np.dot(direction_points[i], tangent[i]))))
    #npr3d_flatten_aligned = AlignPlane(npr3d_flatten, major_axis, minor_axis, normal)
    output_path_plane = f"{OutputFolder}/npr_3d_{i}_aligned.vtp"
    WriteVTPFile(output_path_plane,target_plane)




the number of points in the centerline is: 113
1 (0.788533322672596, -0.591770753017561, 0.167399447155429) [ 0. -1.  0.]
53.71723284864293
3 (0.790714833374902, -0.587410901055853, 0.1723905032235) [ 0. -1.  0.]
54.0265076538936
5 (0.795154144654879, -0.578382396591563, 0.182218795822954) [ 0. -1.  0.]
54.663150720449245
7 (0.802092560363096, -0.563863222502801, 0.196737873626125) [ 0. -1.  0.]
55.67661136328035
9 (0.811803720570109, -0.542755139181487, 0.21538704269434) [ 0. -1.  0.]
57.12860920720156
11 (0.824459469568708, -0.514047703685528, 0.236688701407821) [ 0. -1.  0.]
59.066176603664694
13 (0.839923267036124, -0.477730976705503, 0.257491785087157) [ 0. -1.  0.]
61.4626869042976
15 (0.857527288203128, -0.436513435143393, 0.272218608706871) [ 0. -1.  0.]
64.11836473192247
17 (0.875881580174011, -0.397932175286958, 0.272912882406214) [ 0. -1.  0.]
66.55102774029864
19 (0.892412099256136, -0.374852653738531, 0.25116953056937) [ 0. -1.  0.]
67.98479376663992
21 (0.902026395455656,

In [130]:
# Test

plane = vtk.vtkPlaneSource()
plane.SetOrigin(0.,0.,0.)
#plane.SetNormal(1.,0.,0.)
plane.SetPoint1(0.,1.,0.)
plane.SetPoint2(0.,0.,2.)
plane.Update()
Plane = plane.GetOutput()

WriteVTPFile(f"{OutputFolder}/test_plane_original.vtp", Plane)

transform = vtk.vtkTransform()
transform.RotateX(45)
transform.Update()

transform_filter = vtk.vtkTransformFilter()
transform_filter.SetInputData(Plane)
transform_filter.SetTransform(transform)
transform_filter.Update()
Plane2 = transform_filter.GetOutput()

WriteVTPFile(f"{OutputFolder}/test_plane_transform.vtp", Plane2)

p0 = np.array(Plane.GetPoint(0))
p1 = np.array(Plane.GetPoint(1))
    
current_tangent = p1 - p0
current_tangent /= np.linalg.norm(current_tangent)

T2 = np.array([0,1,0])
B2 = np.array([1,0,0])

axis = np.cross(current_tangent, T2)
axis /= np.linalg.norm(axis)
angle = np.arccos(np.dot(current_tangent, T2))
angle = np.degrees(angle)

if np.dot(axis, B2) < 0:
    angle = -angle


transform = vtk.vtkTransform()
#transform.Translate(-2.,-1.,-3.)
transform.RotateWXYZ(angle, B2[0], B2[1], B2[2])
#transform.Translate(2.,1.,3.)
transform.Update()

transform_filter = vtk.vtkTransformFilter()
transform_filter.SetInputData(Plane)
transform_filter.SetTransform(transform)
transform_filter.Update()
Plane3 = transform_filter.GetOutput()

WriteVTPFile(f"{OutputFolder}/test_plane_transform2.vtp", Plane3)



/var/folders/05/bygg4rdj3n7543jkngl6k6z40000gn/T/ipykernel_1291/2399549486.py:35: RuntimeWarning: invalid value encountered in divide
  axis /= np.linalg.norm(axis)


### Extract the Vessel Cross-section

Pick a point number in the range of the number of points contained in the centerline of the vessel. Adjust the number based on the location of the desired cross-section (proximal/mid/distal).

In [168]:
cs_loc = 25
CrossSection = CutPlane(SphereClip(Volume,path_points[cs_loc],2*a),path_points[cs_loc],direction_points[cs_loc])
CrossSectionGrd = CutPlane(SphereClip(Volume_Gradient,path_points[cs_loc],2*a),path_points[cs_loc],direction_points[cs_loc])

### Save Files

In [84]:
OutputFolder = os.path.dirname(pathline_file)
VesselName = os.path.splitext(os.path.basename(pathline_file))[0]
OutputFolder = os.path.join(OutputFolder,VesselName)

os.system(f"mkdir {OutputFolder}")

mkdir: /Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU03A/SimVascular_Edgard/Paths/L_LAD_0: File exists


256

In [169]:
npr3d_FileName = f"{VesselName}_NPR3D.vtp"
npr2d_FileName = f"{VesselName}_NPR2D.vtp"
#VesselVolume_FileName = f"{VesselName}_Volume.vtu"

WriteVTPFile(os.path.join(OutputFolder,npr3d_FileName), NPR3D)
WriteVTPFile(os.path.join(OutputFolder,npr2d_FileName), NPR2D)
#WriteVTUFile(os.path.join(OutputFolder,VesselVolume_FileName), append_vessel_volume.GetOutput())


In [170]:
CS_FileName = f"{VesselName}_CrossSection_{cs_loc}.vtp"
CS_Grd_FileName = f"{VesselName}_CrossSection_{cs_loc}_Grd.vtp"

WriteVTPFile(os.path.join(OutputFolder,CS_FileName), CrossSection)
WriteVTPFile(os.path.join(OutputFolder,CS_Grd_FileName), CrossSectionGrd)

### Project Vessel Centerline and Cross-Section into 2D

In [171]:
PathFile_vtp = f"{os.path.splitext(pathline_file)[0]}.vtp"
CenterLine = ReadVTPFile(PathFile_vtp)

CenterLine_2D = Transform3Dto2D(CenterLine.GetOutput())
cl_2d_path = os.path.join(OutputFolder,f"{VesselName}_2D.vtp")
WriteVTPFile(cl_2d_path, CenterLine_2D)

In [22]:
CrossSection_2D = Transform3Dto2D(CrossSection)
CS_FileName_2D = f"{VesselName}_CrossSection_{cs_loc}_2D.vtp"
WriteVTPFile(os.path.join(OutputFolder,CS_FileName_2D), CrossSection_2D)

### Cut the Centerline at the Stenosis Location

In [172]:
cs_loc = 25


plane=vtk.vtkPlane()
plane.SetOrigin(path_points[cs_loc])
plane.SetNormal(direction_points[cs_loc])

clipper = vtk.vtkClipDataSet()
clipper.SetInputData(CenterLine.GetOutput()) 
clipper.SetClipFunction(plane)
clipper.GetOutputInformation(1)
clipper.Update()
cl_path_pre = os.path.join(OutputFolder,f"{VesselName}_PreStenosis.vtp")
WriteVTPFile(cl_path_pre,ExtractSurface(clipper.GetOutput()))

clipper.InsideOutOn()
clipper.Update()
cl_path_post = os.path.join(OutputFolder,f"{VesselName}_PostStenosis.vtp")
WriteVTPFile(cl_path_post,ExtractSurface(clipper.GetOutput()))